## Proyecto final BOOTCAMP IRONHACK
###### Estrategias de e-COMERCE

COMPANY = GYMSHARK

### PASO 3. ETL
- website : https://eu.gymshark.com/

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

### DF Nº1: SALES 2023

In [2]:
# Importo el CSV de GYMSHARK SALES resultante del fake
sales_gymshark23_total = pd.read_csv('sales_gymshark23_6np.csv')

In [3]:
len(sales_gymshark23_total)

100000

In [4]:
sales_gymshark23_total.shape

(100000, 12)

In [5]:
sales_gymshark23_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    100000 non-null  int64  
 1   StockCode    100000 non-null  object 
 2   Description  100000 non-null  object 
 3   Size         100000 non-null  object 
 4   Quantity     100000 non-null  int64  
 5   InvoiceDate  100000 non-null  object 
 6   CustomerID   100000 non-null  int64  
 7   Country      100000 non-null  object 
 8   Rating       92719 non-null   float64
 9   Colour       99780 non-null   object 
 10  Price        100000 non-null  float64
 11  Category     100000 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 9.2+ MB


In [6]:
sales_gymshark23_total.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Size', 'Quantity',
       'InvoiceDate', 'CustomerID', 'Country', 'Rating', 'Colour', 'Price',
       'Category'],
      dtype='object')

In [7]:
sales_gymshark23_total.isnull().sum()

InvoiceNo         0
StockCode         0
Description       0
Size              0
Quantity          0
InvoiceDate       0
CustomerID        0
Country           0
Rating         7281
Colour          220
Price             0
Category          0
dtype: int64

In [8]:
# RELLENAR NULOS DE LAS COLUMNAS "Rating" Y "Colour"

# Rellenar valores nulos en la columna "Rating" con 0, son productos que nunca ha sido rankeados por los clientes.
sales_gymshark23_total['Rating'] = sales_gymshark23_total['Rating'].fillna(0)

# Rellenar valores nulos en la columna "Colour" con "multicolored"
sales_gymshark23_total['Colour'] = sales_gymshark23_total['Colour'].fillna('multicolored')


In [9]:
sales_gymshark23_total.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Size           0
Quantity       0
InvoiceDate    0
CustomerID     0
Country        0
Rating         0
Colour         0
Price          0
Category       0
dtype: int64

In [10]:
sales_gymshark23_total.duplicated().sum()

0

In [13]:
# Convertir 'InvoiceDate' a formato de fecha y hora
sales_gymshark23_total['InvoiceDate'] = pd.to_datetime(sales_gymshark23_total['InvoiceDate'], format='%m-%d-%Y %A %H:%M')

# Separar la columna 'InvoiceDate' en fecha y hora
sales_gymshark23_total['Day'] = sales_gymshark23_total['InvoiceDate'].dt.day
sales_gymshark23_total['Month'] = sales_gymshark23_total['InvoiceDate'].dt.month
sales_gymshark23_total['Year'] = sales_gymshark23_total['InvoiceDate'].dt.year
sales_gymshark23_total['Hour'] = sales_gymshark23_total['InvoiceDate'].dt.strftime('%H:%M:%S')
sales_gymshark23_total['DayOfWeek'] = sales_gymshark23_total['InvoiceDate'].dt.dayofweek


# Mostrar las primeras filas
sales_gymshark23_total.head()


,InvoiceNo,StockCode,Description,Size,Quantity,InvoiceDate,CustomerID,Country,Rating,Colour,Price,Category,Day,Month,Year,Hour,DayOfWeek
0,282618,76203V,Apex T-Shirt,L,1,2023-11-22 11:30:00,17849,Australia,4.5,River Stone Grey/Drift Grey,34.04,Men,22,11,2023,11:30:00,2
1,178838,43343R,Pursuit Duffle Bag,M,3,2023-12-25 11:54:00,90379,USA,4.6,Black/White,91.96,Women,25,12,2023,11:54:00,0
2,317707,11168Y,Everyday Mini Backpack,L,1,2023-03-30 09:05:00,73114,UK,0.0,Dusk Green,31.50,Women,30,3,2023,09:05:00,3
3,998032,21561X,No Show Performance Socks,S,6,2023-02-21 15:44:00,21105,USA,5.0,Plum Pink Marl,9.90,Men,21,2,2023,15:44:00,1
4,986803,64146P,GS X Libby Shorts,L,1,2023-03-12 09:29:00,23910,USA,4.2,Camo Brown,44.00,Men,12,3,2023,09:29:00,6


In [14]:
sales_gymshark23_total.to_csv('sales_gymshark_clean2.csv', index=False)

### DF Nº 2: CUSTOMERS 2023

In [15]:
# Importo el CSV de CUSTOMER resultante del fake
customer = pd.read_csv('customer.csv')

In [16]:
len(customer)

100000

In [17]:
customer.shape

(100000, 6)

In [18]:
customer.columns

Index(['CustomerID', 'Name', 'Lastname', 'Age', 'Gender', 'Country'], dtype='object')

In [19]:
customer.isnull().sum()

CustomerID    0
Name          0
Lastname      0
Age           0
Gender        0
Country       0
dtype: int64

In [20]:
customer.duplicated().sum()

0